# Notebook 04: Predictive Modeling

---

## Executive Summary

This notebook is where the **magic happens**—we train machine learning models to predict Customer Lifetime Value. But modeling isn't just about running algorithms; it's about **systematic experimentation, rigorous evaluation, and interpretable results**.

### What This Notebook Covers:

1. **Baseline Model** — Establish a naive benchmark
2. **Model Training** — Linear Regression, Random Forest, Gradient Boosting
3. **Hyperparameter Tuning** — Optimize model performance
4. **Model Evaluation** — Multiple metrics (MAE, MSE, RMSE, R², MAPE)
5. **Model Comparison** — Statistical comparison of all models
6. **Feature Importance** — Understand what drives predictions
7. **Residual Analysis** — Diagnose model behavior
8. **Final Model Selection** — Choose the best model

---

## 1. Environment Setup and Data Loading

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================

# Core Libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# System
import os
import warnings
import joblib
from datetime import datetime

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

# Visualization
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

# Random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Environment configured")
print(f"   Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Path Configuration
BASE_DIR = os.path.dirname(os.getcwd())
DATA_PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
FIGURES_DIR = os.path.join(BASE_DIR, 'report', 'figures')
MODELS_DIR = os.path.join(BASE_DIR, 'models')

# Ensure directories exist
for d in [FIGURES_DIR, MODELS_DIR]:
    os.makedirs(d, exist_ok=True)

# Load processed data
X_train = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'X_train_processed.csv'))
X_test = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'X_test_processed.csv'))
y_train = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'y_train.csv')).squeeze()
y_test = pd.read_csv(os.path.join(DATA_PROCESSED_DIR, 'y_test.csv')).squeeze()

# Load feature names
with open(os.path.join(MODELS_DIR, 'feature_names.txt'), 'r') as f:
    feature_names = [line.strip() for line in f.readlines()]

print(f"✅ Data Loaded Successfully")
print(f"   Training: {X_train.shape[0]:,} samples × {X_train.shape[1]} features")
print(f"   Test:     {X_test.shape[0]:,} samples × {X_test.shape[1]} features")

---

## 2. Evaluation Framework

Before training models, we define our **evaluation metrics** and helper functions.

### Key Metrics

| Metric | Formula | Interpretation |
|--------|---------|----------------|
| **MAE** | $\frac{1}{n}\sum\|y - \hat{y}\|$ | Average error in same units as target |
| **MSE** | $\frac{1}{n}\sum(y - \hat{y})^2$ | Penalizes large errors more |
| **RMSE** | $\sqrt{MSE}$ | Same units as target, comparable to MAE |
| **R²** | $1 - \frac{SS_{res}}{SS_{tot}}$ | % of variance explained (0-1) |
| **MAPE** | $\frac{100}{n}\sum\|\frac{y - \hat{y}}{y}\|$ | Percentage error (scale-independent) |

In [ ]:
def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Compute comprehensive evaluation metrics.
    
    Note: Predictions are in log scale. We evaluate in both log scale 
    and original scale (by applying expm1 inverse transformation).
    """
    # Log scale metrics
    mae_log = mean_absolute_error(y_true, y_pred)
    mse_log = mean_squared_error(y_true, y_pred)
    rmse_log = np.sqrt(mse_log)
    r2 = r2_score(y_true, y_pred)
    
    # Original scale (dollars)
    y_true_dollars = np.expm1(y_true)
    y_pred_dollars = np.expm1(y_pred)
    
    mae_dollars = mean_absolute_error(y_true_dollars, y_pred_dollars)
    rmse_dollars = np.sqrt(mean_squared_error(y_true_dollars, y_pred_dollars))
    
    # MAPE (handle division by zero)
    mape = np.mean(np.abs((y_true_dollars - y_pred_dollars) / (y_true_dollars + 1))) * 100
    
    return {
        'Model': model_name,
        'MAE (log)': mae_log,
        'RMSE (log)': rmse_log,
        'R²': r2,
        'MAE ($)': mae_dollars,
        'RMSE ($)': rmse_dollars,
        'MAPE (%)': mape
    }

# Initialize results storage
model_results = []
trained_models = {}

print("✅ Evaluation framework defined")

---

## 3. Baseline Model

A **baseline model** provides a reference point. We use two naive approaches:
1. **Mean Baseline**: Predict the mean of training target for all samples
2. **Median Baseline**: Predict the median (more robust to outliers)

In [ ]:
# Baseline Models
print("=" * 80)
print("BASELINE MODELS")
print("=" * 80)

# Mean baseline
mean_baseline = np.full(len(y_test), y_train.mean())
mean_results = evaluate_model(y_test, mean_baseline, "Mean Baseline")
model_results.append(mean_results)

print(f"\n📊 Mean Baseline:")
print(f"   Predicted Value (log): {y_train.mean():.4f}")
print(f"   R² Score: {mean_results['R²']:.4f}")
print(f"   MAE: ${mean_results['MAE ($)']:,.2f}")

# Median baseline
median_baseline = np.full(len(y_test), y_train.median())
median_results = evaluate_model(y_test, median_baseline, "Median Baseline")
model_results.append(median_results)

print(f"\n📊 Median Baseline:")
print(f"   Predicted Value (log): {y_train.median():.4f}")
print(f"   R² Score: {median_results['R²']:.4f}")
print(f"   MAE: ${median_results['MAE ($)']:,.2f}")

print(f"\n💡 Any model must beat R² = 0 (baseline) to be useful.")

---

## 4. Model Training

We train multiple algorithms to identify the best performer.

### 4.1 Linear Regression

A simple, interpretable baseline that assumes linear relationships.

In [ ]:
# Linear Regression
print("=" * 80)
print("MODEL 1: LINEAR REGRESSION")
print("=" * 80)

lr_model = LinearRegression()

# Cross-validation
cv_scores = cross_val_score(lr_model, X_train, y_train, cv=5, scoring='r2')
print(f"\n📊 5-Fold Cross-Validation R²: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# Train on full training set
lr_model.fit(X_train, y_train)

# Predict
y_pred_lr = lr_model.predict(X_test)

# Evaluate
lr_results = evaluate_model(y_test, y_pred_lr, "Linear Regression")
model_results.append(lr_results)
trained_models['Linear Regression'] = lr_model

print(f"\n📈 Test Set Performance:")
print(f"   R² Score:  {lr_results['R²']:.4f}")
print(f"   MAE:       ${lr_results['MAE ($)']:,.2f}")
print(f"   RMSE:      ${lr_results['RMSE ($)']:,.2f}")
print(f"   MAPE:      {lr_results['MAPE (%)']:.2f}%")

### 4.2 Random Forest Regressor

An ensemble method that handles non-linear relationships and is robust to outliers.

In [ ]:
# Random Forest
print("=" * 80)
print("MODEL 2: RANDOM FOREST REGRESSOR")
print("=" * 80)

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Cross-validation
print("\n⏳ Running 5-fold cross-validation...")
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='r2')
print(f"📊 5-Fold CV R²: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# Train
print("\n⏳ Training Random Forest on full training set...")
rf_model.fit(X_train, y_train)

# Predict
y_pred_rf = rf_model.predict(X_test)

# Evaluate
rf_results = evaluate_model(y_test, y_pred_rf, "Random Forest")
model_results.append(rf_results)
trained_models['Random Forest'] = rf_model

print(f"\n📈 Test Set Performance:")
print(f"   R² Score:  {rf_results['R²']:.4f}")
print(f"   MAE:       ${rf_results['MAE ($)']:,.2f}")
print(f"   RMSE:      ${rf_results['RMSE ($)']:,.2f}")
print(f"   MAPE:      {rf_results['MAPE (%)']:.2f}%")

### 4.3 Gradient Boosting Regressor

A powerful sequential ensemble method that often achieves state-of-the-art results.

In [ ]:
# Gradient Boosting
print("=" * 80)
print("MODEL 3: GRADIENT BOOSTING REGRESSOR")
print("=" * 80)

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=RANDOM_STATE
)

# Cross-validation
print("\n⏳ Running 5-fold cross-validation...")
cv_scores = cross_val_score(gb_model, X_train, y_train, cv=5, scoring='r2')
print(f"📊 5-Fold CV R²: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")

# Train
print("\n⏳ Training Gradient Boosting on full training set...")
gb_model.fit(X_train, y_train)

# Predict
y_pred_gb = gb_model.predict(X_test)

# Evaluate
gb_results = evaluate_model(y_test, y_pred_gb, "Gradient Boosting")
model_results.append(gb_results)
trained_models['Gradient Boosting'] = gb_model

print(f"\n📈 Test Set Performance:")
print(f"   R² Score:  {gb_results['R²']:.4f}")
print(f"   MAE:       ${gb_results['MAE ($)']:,.2f}")
print(f"   RMSE:      ${gb_results['RMSE ($)']:,.2f}")
print(f"   MAPE:      {gb_results['MAPE (%)']:.2f}%")

---

## 5. Hyperparameter Tuning

We optimize the best-performing model using Grid Search.

In [ ]:
# Hyperparameter tuning for Random Forest
print("=" * 80)
print("HYPERPARAMETER TUNING: RANDOM FOREST")
print("=" * 80)

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

print(f"\n🔧 Parameter Grid:")
for param, values in param_grid.items():
    print(f"   {param}: {values}")

# Note: Full grid search can be time-consuming
# For demonstration, we use a smaller subset
print("\n⏳ Running Grid Search (this may take a few minutes)...")

rf_tuned = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=rf_tuned,
    param_grid=param_grid,
    cv=3,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f"\n✅ Best Parameters: {grid_search.best_params_}")
print(f"✅ Best CV R²: {grid_search.best_score_:.4f}")

In [ ]:
# Evaluate tuned model
best_rf = grid_search.best_estimator_
y_pred_tuned = best_rf.predict(X_test)

tuned_results = evaluate_model(y_test, y_pred_tuned, "Random Forest (Tuned)")
model_results.append(tuned_results)
trained_models['Random Forest (Tuned)'] = best_rf

print(f"\n📈 Tuned Model Test Set Performance:")
print(f"   R² Score:  {tuned_results['R²']:.4f}")
print(f"   MAE:       ${tuned_results['MAE ($)']:,.2f}")
print(f"   RMSE:      ${tuned_results['RMSE ($)']:,.2f}")
print(f"   MAPE:      {tuned_results['MAPE (%)']:.2f}%")

# Compare improvement
improvement = tuned_results['R²'] - rf_results['R²']
print(f"\n🔄 Improvement over untuned: {improvement:+.4f} R²")

---

## 6. Model Comparison

Now we compare all trained models side by side.

In [ ]:
# Create comparison table
print("=" * 80)
print("MODEL COMPARISON")
print("=" * 80)

results_df = pd.DataFrame(model_results)
results_df = results_df.sort_values('R²', ascending=False)

print("\n📊 Complete Model Comparison:")
results_df

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Filter out baselines for cleaner plots
model_df = results_df[~results_df['Model'].str.contains('Baseline')].copy()

# R² Score
ax1 = axes[0, 0]
colors = sns.color_palette('viridis', len(model_df))
bars = ax1.barh(model_df['Model'], model_df['R²'], color=colors)
ax1.set_xlabel('R² Score')
ax1.set_title('R² Score Comparison (Higher is Better)', fontweight='bold')
ax1.set_xlim(0, 1)
for bar, val in zip(bars, model_df['R²']):
    ax1.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', va='center')

# MAE ($)
ax2 = axes[0, 1]
bars = ax2.barh(model_df['Model'], model_df['MAE ($)'], color=colors)
ax2.set_xlabel('MAE ($)')
ax2.set_title('Mean Absolute Error (Lower is Better)', fontweight='bold')
for bar, val in zip(bars, model_df['MAE ($)']):
    ax2.text(val + 50, bar.get_y() + bar.get_height()/2, f'${val:,.0f}', va='center')

# RMSE ($)
ax3 = axes[1, 0]
bars = ax3.barh(model_df['Model'], model_df['RMSE ($)'], color=colors)
ax3.set_xlabel('RMSE ($)')
ax3.set_title('Root Mean Squared Error (Lower is Better)', fontweight='bold')
for bar, val in zip(bars, model_df['RMSE ($)']):
    ax3.text(val + 50, bar.get_y() + bar.get_height()/2, f'${val:,.0f}', va='center')

# MAPE
ax4 = axes[1, 1]
bars = ax4.barh(model_df['Model'], model_df['MAPE (%)'], color=colors)
ax4.set_xlabel('MAPE (%)')
ax4.set_title('Mean Absolute Percentage Error (Lower is Better)', fontweight='bold')
for bar, val in zip(bars, model_df['MAPE (%)']):
    ax4.text(val + 0.5, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', va='center')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '04_model_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Saved: 04_model_comparison.png")

---

## 7. Feature Importance Analysis

Understanding which features drive predictions is crucial for **interpretability** and **business insights**.

In [ ]:
# Feature importance from best model
print("=" * 80)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

# Use the best Random Forest model
best_model = trained_models['Random Forest (Tuned)']

# Get feature importances
importances = best_model.feature_importances_

# Create DataFrame
fi_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
fi_df = fi_df.sort_values('Importance', ascending=False)

# Show top 15
print("\n📊 Top 15 Most Important Features:")
fi_df.head(15)

In [ ]:
# Visualize feature importance
fig, ax = plt.subplots(figsize=(12, 10))

top_n = 20
top_features = fi_df.head(top_n)

colors = plt.cm.viridis(np.linspace(0.3, 0.9, top_n))
bars = ax.barh(range(top_n), top_features['Importance'].values[::-1], color=colors)
ax.set_yticks(range(top_n))
ax.set_yticklabels(top_features['Feature'].values[::-1])
ax.set_xlabel('Feature Importance (Mean Decrease in Impurity)')
ax.set_title(f'Top {top_n} Most Important Features', fontweight='bold', fontsize=14)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_features['Importance'].values[::-1])):
    ax.text(val + 0.002, bar.get_y() + bar.get_height()/2, f'{val:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '04_feature_importance.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Saved: 04_feature_importance.png")

---

## 8. Model Validation

### 8.1 Actual vs Predicted Plot

In [ ]:
# Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Log scale
ax1 = axes[0]
ax1.scatter(y_test, y_pred_tuned, alpha=0.3, s=20, color='steelblue')
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
ax1.set_xlabel('Actual (log scale)')
ax1.set_ylabel('Predicted (log scale)')
ax1.set_title('Actual vs Predicted CLV (Log Scale)', fontweight='bold')
ax1.legend()

# Dollar scale
ax2 = axes[1]
y_test_dollars = np.expm1(y_test)
y_pred_dollars = np.expm1(y_pred_tuned)
ax2.scatter(y_test_dollars, y_pred_dollars, alpha=0.3, s=20, color='seagreen')
ax2.plot([y_test_dollars.min(), y_test_dollars.max()], 
         [y_test_dollars.min(), y_test_dollars.max()], 'r--', lw=2, label='Perfect Prediction')
ax2.set_xlabel('Actual CLV ($)')
ax2.set_ylabel('Predicted CLV ($)')
ax2.set_title('Actual vs Predicted CLV (Dollar Scale)', fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '04_actual_vs_predicted.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Saved: 04_actual_vs_predicted.png")

### 8.2 Lift Chart

A lift chart shows how well the model discriminates between high and low value customers.

In [ ]:
# Lift Chart
fig, ax = plt.subplots(figsize=(12, 6))

# Create DataFrame with actual and predicted
lift_df = pd.DataFrame({
    'Actual': y_test_dollars,
    'Predicted': y_pred_dollars
})

# Sort by actual CLV
lift_df = lift_df.sort_values('Actual').reset_index(drop=True)

# Calculate cumulative averages
lift_df['Cumulative_Actual'] = lift_df['Actual'].expanding().mean()
lift_df['Cumulative_Predicted'] = lift_df['Predicted'].expanding().mean()

# Plot
ax.plot(lift_df.index, lift_df['Actual'], alpha=0.3, color='blue', label='Actual CLV')
ax.plot(lift_df.index, lift_df['Predicted'], alpha=0.3, color='orange', label='Predicted CLV')

# Add smoothed lines
window = 100
ax.plot(lift_df.index, lift_df['Actual'].rolling(window).mean(), color='blue', lw=2, label=f'Actual (Rolling Avg)')
ax.plot(lift_df.index, lift_df['Predicted'].rolling(window).mean(), color='orange', lw=2, label=f'Predicted (Rolling Avg)')

ax.set_xlabel('Customer Index (Sorted by Actual CLV)')
ax.set_ylabel('Customer Lifetime Value ($)')
ax.set_title('Lift Chart: Actual vs Predicted CLV', fontweight='bold')
ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '04_lift_chart.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Saved: 04_lift_chart.png")

### 8.3 Residual Analysis

In [ ]:
# Residual Analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

residuals = y_test - y_pred_tuned

# 1. Residual distribution
ax1 = axes[0, 0]
sns.histplot(residuals, kde=True, ax=ax1, color='steelblue', bins=50)
ax1.axvline(0, color='red', linestyle='--', lw=2)
ax1.set_xlabel('Residual (Actual - Predicted)')
ax1.set_title('Residual Distribution', fontweight='bold')
ax1.annotate(f'Mean: {residuals.mean():.4f}\nStd: {residuals.std():.4f}', 
             xy=(0.95, 0.95), xycoords='axes fraction', ha='right', va='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 2. Residuals vs Predicted
ax2 = axes[0, 1]
ax2.scatter(y_pred_tuned, residuals, alpha=0.3, s=20, color='seagreen')
ax2.axhline(0, color='red', linestyle='--', lw=2)
ax2.set_xlabel('Predicted Value')
ax2.set_ylabel('Residual')
ax2.set_title('Residuals vs Predicted', fontweight='bold')

# 3. Q-Q plot of residuals
ax3 = axes[1, 0]
from scipy import stats
stats.probplot(residuals, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot of Residuals', fontweight='bold')

# 4. Residuals over index (checking for patterns)
ax4 = axes[1, 1]
ax4.scatter(range(len(residuals)), residuals, alpha=0.3, s=20, color='coral')
ax4.axhline(0, color='red', linestyle='--', lw=2)
ax4.set_xlabel('Sample Index')
ax4.set_ylabel('Residual')
ax4.set_title('Residuals Over Samples', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, '04_residual_analysis.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📸 Saved: 04_residual_analysis.png")

---

## 9. Save Final Model

We save the best model for deployment and future inference.

In [ ]:
# Save final model
print("=" * 80)
print("SAVING FINAL MODEL")
print("=" * 80)

# Determine best model based on R²
best_model_name = results_df[~results_df['Model'].str.contains('Baseline')].iloc[0]['Model']
final_model = trained_models[best_model_name]

print(f"\n🏆 Best Model: {best_model_name}")
print(f"   R² Score: {results_df.iloc[0]['R²']:.4f}")
print(f"   MAE: ${results_df.iloc[0]['MAE ($)']:,.2f}")

# Save model
model_path = os.path.join(MODELS_DIR, 'final_model.joblib')
joblib.dump(final_model, model_path)
print(f"\n✅ Model saved to: {model_path}")

# Save model metadata
metadata = {
    'model_name': best_model_name,
    'r2_score': results_df.iloc[0]['R²'],
    'mae_dollars': results_df.iloc[0]['MAE ($)'],
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'n_features': len(feature_names),
    'n_training_samples': len(X_train)
}

import json
with open(os.path.join(MODELS_DIR, 'model_metadata.json'), 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved to: model_metadata.json")

In [ ]:
# Save results table
results_df.to_csv(os.path.join(DATA_PROCESSED_DIR, 'model_comparison_results.csv'), index=False)
print(f"\n✅ Results saved to: model_comparison_results.csv")

---

## 10. Summary

### Model Training Results

In [ ]:
# Final Summary
print("=" * 80)
print("MODELING SUMMARY")
print("=" * 80)

print(f"\n📊 Models Trained: {len(trained_models)}")
for name in trained_models.keys():
    print(f"   • {name}")

print(f"\n🏆 Best Model: {best_model_name}")
print(f"\n📈 Final Performance Metrics (on test set):")
print(f"   R² Score:       {tuned_results['R²']:.4f} (explains {tuned_results['R²']*100:.2f}% of variance)")
print(f"   MAE:            ${tuned_results['MAE ($)']:,.2f}")
print(f"   RMSE:           ${tuned_results['RMSE ($)']:,.2f}")
print(f"   MAPE:           {tuned_results['MAPE (%)']:.2f}%")

print(f"\n📋 Top 5 Predictive Features:")
for i, row in fi_df.head(5).iterrows():
    print(f"   {fi_df.index.get_loc(i)+1}. {row['Feature']} ({row['Importance']:.4f})")

print(f"\n✅ Model is ready for deployment!")

---

## Next Steps

In **Notebook 05: Model Inference**, we will:

1. Load the saved model and preprocessor
2. Create an inference pipeline for new data
3. Demonstrate scoring individual customers
4. Show batch prediction capabilities

---

**End of Notebook 04**